<a href="https://colab.research.google.com/github/ghanendrapiplani/ml_projects/blob/master/tflow1_linearreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [6]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [7]:
rnk1_tensor = tf.Variable(["a",'b'], tf.string)
rnk2_tensor = tf.Variable([["a",'b',"f"],["c",'d',"g"]], tf.string)
print(tf.rank(rnk1_tensor))
print(tf.rank(rnk2_tensor))

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


In [8]:
print(tf.shape(rnk1_tensor))
print(tf.shape(rnk2_tensor))

tf.Tensor([2], shape=(1,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)


In [9]:
tensor1 = tf.ones([1, 2, 3])
print(tensor1)

tf.Tensor(
[[[1. 1. 1.]
  [1. 1. 1.]]], shape=(1, 2, 3), dtype=float32)


In [10]:
tensor2 = tf.reshape(tensor1, [2, 3, 1])
print(tensor2)
tensor3 = tf.reshape(tensor1, [3, 2, 1])
print(tensor3)

tf.Tensor(
[[[1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]]], shape=(2, 3, 1), dtype=float32)
tf.Tensor(
[[[1.]
  [1.]]

 [[1.]
  [1.]]

 [[1.]
  [1.]]], shape=(3, 2, 1), dtype=float32)


In [0]:
!pip install -q sklearn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [13]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
# print(dftrain.head)
# print(dfeval.head)
y_train = dftrain.pop("survived")
y_eval = dfeval.pop("survived")
print(y_train.head)
print(dftrain.head)

<bound method NDFrame.head of 0      0
1      1
2      1
3      1
4      0
      ..
622    0
623    0
624    1
625    0
626    0
Name: survived, Length: 627, dtype: int64>
<bound method NDFrame.head of         sex   age  n_siblings_spouses  ...     deck  embark_town alone
0      male  22.0                   1  ...  unknown  Southampton     n
1    female  38.0                   1  ...        C    Cherbourg     n
2    female  26.0                   0  ...  unknown  Southampton     y
3    female  35.0                   1  ...        C  Southampton     n
4      male  28.0                   0  ...  unknown   Queenstown     y
..      ...   ...                 ...  ...      ...          ...   ...
622    male  28.0                   0  ...  unknown  Southampton     y
623    male  25.0                   0  ...  unknown  Southampton     y
624  female  19.0                   0  ...        B  Southampton     y
625  female  28.0                   1  ...  unknown  Southampton     n
626    male  32.0

In [14]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_cols = []

for cat_col in CATEGORICAL_COLUMNS :
  vocab = dftrain[cat_col].unique()
  feature_cols.append(tf.feature_column.categorical_column_with_vocabulary_list(cat_col,vocab))

for num_col in NUMERIC_COLUMNS:
  feature_cols.append(tf.feature_column.numeric_column(num_col,dtype=tf.float32))

print(feature_cols)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [15]:
def input_func(data_df, label_df, num_epochs=20, shuffle=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds


train_input_fn = lambda: input_func(dftrain, y_train)
eval_input_fn = lambda: input_func(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_cols)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

0.7651515


In [57]:
pred_dict = list(linear_est.predict(eval_input_fn))
for i in range(len(pred_dict)):
  print("\n\n")
  print("Model prediction = "+str(pred_dict[i]['probabilities'][1]*100.0))
  print("True status = "+str(y_eval.loc[i]))
  print("Person stats = \n"+str(dftrain.loc[i][0]))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpr_77mjhx/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.



Model prediction = 4.633549973368645
True status = 0
Person stats = 
male



Model prediction = 28.570720553398132
True status = 0
Person stats = 
female



Model prediction = 62.80483603477478
True status = 1
Person stats = 
female



Mo